In [62]:
!pip install langchain langchain-openai langchain_chroma langchain-text-splitters langchain_community langchainhub

  Using cached langchain-0.2.11-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_openai-0.1.19-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached langchain_community-0.2.10-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchainhub-0.1.20-py3-none-any.whl.metadata (659 bytes)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached langchain_core-0.2.24-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.93-py3-none-any.whl.metadata (13 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached openai-1.37.1-py3-none-any.whl.metadata (22 kB)
  Using cached chromadb-0.5.5-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.111.1-py3-none-any.whl.metadata (26 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Usin

In [1]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = 'sk-'

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [59]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [40]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=([
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

relevance_prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
retriever1 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
retriever2 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

# Only retrieve documents that have a relevance score
# Above a certain threshold
retriever3 = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)

# Only get the single most similar document from the dataset
retriever4 = vectorstore.as_retriever(search_kwargs={'k': 1})

# Use a filter to only retrieve documents from a specific paper
# vectorstore.as_retriever(
#     search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
# )

retrievers = [retriever1, retriever2, retriever3, retriever4]

In [62]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableBranch
from langchain_core.output_parsers import JsonOutputParser
import json

relevance_prompt_template = """
You are given a context and a question. Evaluate the relevance of the context to the question.
If the context is relevant to the question, return {{"relevance": "yes"}}.
If the context is not relevant to the question, return {{"relevance": "no"}}.

Context: {context}
Question: {question}
Answer format: json
"""

# Chain for relevance evaluation
relevance_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | RunnableLambda(lambda inputs: {
        "context": format_docs(inputs["context"]["context"]),
        "question": inputs["question"]
    })
    | RunnableLambda(lambda inputs: relevance_prompt_template.format(context=inputs["context"], question=inputs["question"]))
    | llm
    | RunnableLambda(lambda response: response.content)
    | JsonOutputParser()
)

# Combine retrievers and evaluate relevance
def retrieve_and_evaluate(query):
    results = {}
    for i, retriever in enumerate(retrievers):
        retrieved_docs = retriever.get_relevant_documents(query)
        evaluated_relevance = relevance_chain.invoke({"context": retrieved_docs, "question": query})
        if evaluated_relevance["relevance"] == "yes":
            return retriever
    return None

branch = RunnableBranch(
    lambda result: result is None,
    if_true=RunnableLambda(lambda _: "No"),
    if_false=(
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
        | RunnableLambda(lambda inputs: f"Context: {inputs['context']}\n\nQuestion: {inputs['question']}")
        | llm
    )
)

rag_chain = (
    {"context": RunnableLambda(retrieve_and_evaluate) | format_docs, "question": RunnablePassthrough()}
    | prompt
    | branch
    | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(response)

TypeError: RunnableBranch.__init__() got an unexpected keyword argument 'if_true'